In [41]:
import os 
import pandas as pd
import shutil
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold

seed = 5
# directories
dirpath = os.getcwd()
img_dir = os.path.join(os.path.normpath(dirpath + os.sep + os.pardir), r'udder_processing\validate_watershed\watershed_data\masked_frame')
arr_dir =  os.path.join(os.path.normpath(dirpath + os.sep + os.pardir), r'udder_processing\validate_watershed\watershed_data\array')
msk_dir =  os.path.join(os.path.normpath(dirpath + os.sep + os.pardir), r'udder_processing\validate_watershed\watershed_data\ws_mask')
data2_dir = os.path.join(dirpath, r"ws_classify\array\folds")
data1_dir = os.path.join(dirpath, r"ws_classify\masked_frame\folds")
data3_dir = os.path.join(dirpath, r"ws_classify\ws_mask\folds")
# class sest df
class_df = pd.read_csv(os.path.join(os.path.normpath(dirpath + os.sep + os.pardir), r'udder_processing\validate_watershed\survey_results.csv'))
class_dict = {0:"bad", 1:"good"}

def mk_dest_dir(data_dir, fold, file_set, file_class):
    if not os.path.exists(data_dir):
        os.mkdir(data_dir)
    fold_dir = os.path.join(data_dir,fold)
    if not os.path.exists(fold_dir):
        os.mkdir(fold_dir)
    set_dir = os.path.join(fold_dir, file_set)
    if not os.path.exists(set_dir):
        os.mkdir(set_dir)
    class_dir = os.path.join(set_dir, file_class)
    if not os.path.exists(class_dir):
        os.mkdir(class_dir)

In [42]:
splits = 5
fold_df = pd.DataFrame(columns=["cow", "fold_group"])
x = np.array(list(class_df.filename))
group_cows = np.array(list(class_df.cow))
group_class = np.array(list(class_df.img_class))
sgkf = StratifiedGroupKFold(n_splits=5)
sgkf.get_n_splits(x, group_class)
test_list = []
fold_list = []
for i, (train_index, test_index) in enumerate(sgkf.split(x, group_class, group_cows)):
    test_cows = np.unique(group_cows[test_index])
    test_list.extend(list(test_cows))
    fold_list.extend([i]*len(test_cows))
temp_df = pd.DataFrame({"cow": test_list, "fold_group": fold_list})
fold_df = pd.concat([fold_df , temp_df], axis = 0, ignore_index = True)
    
merged = pd.merge(class_df, fold_df, on = "cow")



In [43]:
split_groups = list(range(splits))
set_folds_df = pd.DataFrame()
for group in split_groups:
    non_test = split_groups.copy()
    non_test.remove(group)
    np.random.seed(seed+group)
    np.random.shuffle(non_test)
    all_test = non_test +[group]
    set_list = ["train"] * 3 + ["val"] +["test"]
    set_df = pd.DataFrame({"fold_group":all_test, "kfold":group, "set_name": set_list})
    set_folds_df = pd.concat([set_folds_df, set_df], axis =0, ignore_index = True)
merged2 = pd.merge(merged, set_folds_df, on = "fold_group")
merged2.to_csv("wsclass_sets.csv", index = False)

In [44]:
merged2

,cow,filename,img_class,fold_group,kfold,set_name
0,1057,1057_20231117_140050_frame_621,1,2,0,train
1,1057,1057_20231117_140050_frame_621,1,2,1,train
2,1057,1057_20231117_140050_frame_621,1,2,2,test
3,1057,1057_20231117_140050_frame_621,1,2,3,train
4,1057,1057_20231117_140050_frame_621,1,2,4,val
...,...,...,...,...,...,...
1495,1003,1003_20231117_163646_frame_264,1,3,0,train
1496,1003,1003_20231117_163646_frame_264,1,3,1,val
1497,1003,1003_20231117_163646_frame_264,1,3,2,train
1498,1003,1003_20231117_163646_frame_264,1,3,3,test


In [3]:
# merged_group = merged[["fold_group", "cow", "img_class"]].groupby(["fold_group", "cow"]).agg(["sum", "count"]).reset_index()
# merged_group.columns = ["_".join(name) if len(name[1]) >1 else name[0] for name in merged_group.columns]
# merged_group.sort_values(by =["fold_group", "cow"]).reset_index(drop = True)

In [29]:
# verify cows do not reapeatd
train_cows = set(merged2[merged2.set_name == "train"]["cow"].values)
test_cows = set(merged2[merged2.set_name == "test"]["cow"].values)
val_cows = set(merged2[merged2.set_name == "val"]["cow"].values)
print(val_cows.intersection(test_cows, train_cows))
print(train_cows.intersection(test_cows, val_cows))
print(test_cows.intersection(val_cows, train_cows))

{1288, 778, 1291, 915, 1300, 1057, 929, 1315, 808, 1320, 1325, 948, 837, 1223, 1354, 973, 975, 1360, 978, 723, 1236, 981, 729, 1116, 868, 1003, 1262, 1271}
{1288, 778, 1291, 915, 1300, 1057, 929, 1315, 808, 1320, 1325, 948, 837, 1223, 1354, 973, 975, 1360, 978, 723, 1236, 981, 729, 1116, 868, 1003, 1262, 1271}
{1288, 778, 1291, 915, 1300, 1057, 929, 1315, 808, 1320, 1325, 948, 837, 1223, 1354, 973, 975, 1360, 978, 723, 1236, 981, 729, 1116, 868, 1003, 1262, 1271}


In [45]:
# class sest df
sets_df = pd.read_csv("wsclass_sets.csv")

In [55]:
for file in range(len(sets_df.filename)):
    file_line = sets_df.iloc[file]
    name = file_line.filename
    src3 = os.path.join(msk_dir, name +".png")
    src1 = os.path.join(img_dir, name +".png")
    src2 = os.path.join(arr_dir, name +".npy")
    fold =  "data_k"+ str(file_line["kfold"])
    # find/create dest directoy
    file_set = file_line["set_name"]
    file_class = class_dict[file_line["img_class"]]
    mk_dest_dir(data1_dir, fold, file_set, file_class)
    mk_dest_dir(data2_dir, fold, file_set, file_class)
    mk_dest_dir(data3_dir, fold, file_set, file_class)
    dest1_dir = os.path.join(data1_dir,fold , file_set, file_class)
    dest2_dir = os.path.join(data2_dir,fold, file_set, file_class)
    dest3_dir = os.path.join(data3_dir,fold, file_set, file_class)
    # print(dest1_dir)
    shutil.copy(src1, dest1_dir)
    shutil.copy(src2, dest2_dir)
    shutil.copy(src3, dest3_dir)